# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [132]:
import pandas as pd

---
## 2. Load data

In [133]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [134]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.000000,...,0.000131,0.000041,0.000908,2.086294,99.530517,44.235794,2.086425,99.530558,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.950000,...,0.000003,0.001218,0.000000,0.009482,0.000000,0.000000,0.009485,0.001218,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.960000,...,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000004,0.000000,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [135]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.000000,0.000000,44.266931,0.000000,0.000000
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.000000,0.000000,44.266931,0.000000,0.000000
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.000000,0.000000,44.266931,0.000000,0.000000
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.000000,0.000000,44.266931,0.000000,0.000000
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.000000,0.000000,44.266931,0.000000,0.000000


In [136]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as you can!

In [137]:
import numpy as np

def add_contract_features(df, today=None):
    """
     Adds contract-related features for churn prediction, including tenure, contract timing,
    renewal flags, and tenure buckets.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Must contain columns: 'contract_start_date', 'contract_end_date', 'next_renewal_date'
    today : str or datetime (optional)
        Reference date to calculate tenure (default = today)
    """
    
    # ensure datetime format
    df = df.copy()
    df['date_activ'] = pd.to_datetime(df['date_activ'])
    df['date_end'] = pd.to_datetime(df['date_end'])
    df['date_renewal'] = pd.to_datetime(df['date_renewal'])
    
    if today is None:
        today = pd.Timestamp.today().normalize()
    else:
        today = pd.to_datetime(today)
    
    # Feature 1: Antiquity (tenure) in months
    df['tenure_months'] = (today - df['date_activ']).dt.days // 30
    
    # Feature 2: Remaining time until contract end (in months)
    df['time_to_end_months'] = (df['date_end'] - today).dt.days // 30

    # Time until contract end and renewal
    df['time_to_end_months_raw'] = (df['date_end'] - today).dt.days // 30
    df['time_to_renewal_months_raw'] = (df['date_renewal'] - today).dt.days // 30
    
    # Flags for expired/missed
    df['contract_expired_flag'] = (df['time_to_end_months_raw'] < 0).astype(int)
    df['renewal_missed_flag'] = (df['time_to_renewal_months_raw'] < 0).astype(int)
    # Feature 3: Time until next renewal (in months)
    df['time_to_renewal_months'] = (df['date_renewal'] - today).dt.days // 30
    
    # Clip negatives to 0 for modeling
    df['time_to_end_months'] = df['time_to_end_months_raw'].clip(lower=0)
    df['time_to_renewal_months'] = df['time_to_renewal_months_raw'].clip(lower=0)
    
        # Feature 4: Contract length (total months)
    df['contract_length_months'] = (df['date_end'] - df['date_activ']).dt.days // 30
    
    # Feature 5: Renewal indicator (is renewal due within next 3 months?)
    df['renewal_due_soon'] = (df['time_to_renewal_months'] <= 3).astype(int)

    # Feature 5:tenure buckets
    df['tenure_bucket'] = pd.cut(df['tenure_months'],
                             bins=[0, 6, 24, 60, 120, np.inf],   # Added 60–120 and 120+ bins
                             labels=['new', 'mid', 'established', 'loyal', 'very_loyal'])

    # Drop raw intermediate columns if desired (optional)
    df.drop(['time_to_end_months_raw', 'time_to_renewal_months_raw'], axis=1, inplace=True)
    return df


In [138]:
'''Flags contract_expired_flag and renewal_missed_flag capture negative timing information.

Numeric columns time_to_end_months and time_to_renewal_months are clipped at 0 for modeling.

Tenure bucket handles long-term customers automatically.

Tenure (antiquity) → new customers churn more, long-tenure customers are stable.

Time to contract end → customers often switch at contract expiry.

Time to renewal → approaching renewal is a churn risk point.

Contract length → shorter contracts = higher churn risk.

Renewal due soon flag → simple, interpretable signal.'''


#df = df.copy()
 
contract_features = add_contract_features(df[['id', 'date_activ', 'date_end', 'date_renewal']])
df = df.merge(
    contract_features[['id', 'tenure_months', 'tenure_bucket','time_to_end_months', 
                       'contract_expired_flag','time_to_renewal_months','renewal_missed_flag']],
    on='id',
    how='left'
)

df.head(3)


,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn,tenure_months,tenure_bucket,time_to_end_months,contract_expired_flag,time_to_renewal_months,renewal_missed_flag
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.000000,...,2.086425,99.530558,44.236702,1,149,very_loyal,0,1,0,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.950000,...,0.009485,0.001218,0.000000,0,196,very_loyal,0,1,0,1
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.960000,...,0.000004,0.000000,0.000000,0,188,very_loyal,0,1,0,1


In [139]:
def add_consumption_features(df):
    """
    Adds electricity & gas consumption–based churn features.
    Requires columns:
        - yearly_total_cons
        - last_month_cos
        - yearly_gas_cons
    """
    import numpy as np
    df = df.copy()

    # --- Electricity behaviour ---
    # average monthly electricity use
    df['avg_monthly_cons'] = df['cons_12m'] / 12

    # ratio: how last month's use compares to usual use
    df['elec_usage_ratio'] = np.where(
    df['avg_monthly_cons'] > 0,
    df['cons_last_month'] / df['avg_monthly_cons'],
    np.nan
    )

    # Fill missing or invalid ratios with median
    median_ratio = df['elec_usage_ratio'].median()
    df['elec_usage_ratio'] = df['elec_usage_ratio'].fillna(median_ratio)
    

    # flag: large drop in electricity usage (<50% of normal)
    df['elec_drop_flag'] = (df['elec_usage_ratio'] < 0.5).astype(int)

    # --- Gas behaviour ---
    # normalize gas consumption (kg/therm etc.)
    df['avg_monthly_gas'] = df['cons_gas_12m'] / 12

    # since you don’t have last_month_gas, you can still use yearly_gas_total itself
    # to capture very low-usage customers (who often churn)
    df['low_gas_user_flag'] = (df['cons_gas_12m'] < df['cons_gas_12m'].median()).astype(int)

    return df


In [140]:

'''df.drop(columns=['avg_monthly_cons_x','elec_usage_ratio_x','elec_drop_flag_x','avg_monthly_gas_x','low_gas_user_flag_x',
                 'avg_monthly_cons_y','elec_usage_ratio_y','elec_drop_flag_y','avg_monthly_gas_y','low_gas_user_flag_y',
                 'avg_monthly_cons','elec_usage_ratio','elec_drop_flag','avg_monthly_gas','low_gas_user_flag'],
                           )'''
#df.head(3)
cons_features = add_consumption_features(df[['id','cons_12m','cons_last_month','cons_gas_12m']])
df = df.merge(cons_features[['id','avg_monthly_cons','elec_usage_ratio','elec_drop_flag','avg_monthly_gas','low_gas_user_flag']],
                           on='id', how='left')

df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,tenure_bucket,time_to_end_months,contract_expired_flag,time_to_renewal_months,renewal_missed_flag,avg_monthly_cons,elec_usage_ratio,elec_drop_flag,avg_monthly_gas,low_gas_user_flag
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.000000,...,very_loyal,0,1,0,1,0.000000,0.873417,0,4578.833333,0
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.950000,...,very_loyal,0,1,0,1,388.333333,0.000000,1,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.960000,...,very_loyal,0,1,0,1,45.333333,0.000000,1,0.000000,0


In [141]:
'''(a) Ratio-based features

Compare paid vs. actual consumption:

df['paid_to_total_ratio'] = df['paid_consumption'] / df['cons_12m']


If ratio ≈ 1 → customer has paid for almost everything (healthy behavior).

If ratio ≪ 1 → might be lagging or inactive (possible churn).

 (b) Payment lag proxy

If you have monthly data, you can compute a rolling ratio:

df['payment_gap_flag'] = (df['paid_consumption'] < 0.8 * df['cons_12m']).astype(int)


This indicates customers who have not paid at least 80% of their yearly usage — potential churn or credit risk.

 (c) Normalized consumption per payment

To standardize across customers:

df['avg_unit_value'] = df['paid_consumption'] / (df['yearly_total_kwh'] + 1e-6)


→ 
 (e) Combined score

You can combine usage and payment info into a single engagement score:

df['engagement_score'] = (
    0.5 * df['paid_to_total_ratio'] +
    0.3 * (df['last_month_kwh'] / (df['yearly_total_kwh']/12)) +
    0.2 * (1 - df['elec_drop_flag'])
)


Higher score → more active & paying customers → less likely to churn'''



def add_current_paid_consumption_features(df):
    
    df = df.copy()

    # --- Basic sanity checks ---
    df['cons_12m'] = df['cons_12m'].replace(0, np.nan)
    df['avg_monthly_cons1'] = df['cons_12m'] / 12

    # --- Paid vs yearly total ratio ---
    df['paid_to_total_ratio'] = df['imp_cons'] / df['cons_12m']
    df['paid_to_total_ratio'] = df['paid_to_total_ratio'].replace([np.inf, -np.inf], np.nan)
    df['paid_to_total_ratio'] = df['paid_to_total_ratio'].fillna(df['paid_to_total_ratio'].median())


    # --- Paid vs avg monthly consumption ---
    df['paid_to_avg_month_ratio'] = df['imp_cons'] / df['avg_monthly_cons1']

    # --- Paid vs last month usage (short-term behavior) ---
    df['paid_to_last_month_ratio'] = df['imp_cons'] / df['cons_last_month']

    df['last_month_ratio'] = df['cons_last_month'] / df['avg_monthly_cons1']
    df['last_month_ratio'] = df['last_month_ratio'].replace([np.inf, -np.inf], np.nan)
    df['last_month_ratio'] = df['last_month_ratio'].fillna(df['last_month_ratio'].median())

    # ---Payment lag flag ---
    df['payment_lag_flag'] = (df['paid_to_total_ratio'] < 0.5).astype(int)

    # --- engagement_score  ---
    df['engagement_score'] = (
    0.5 * df['paid_to_total_ratio'] +
    0.3 * (df['last_month_ratio']) +
    0.2 * (1 - df['elec_drop_flag'])
     )

    # --- Missing/invalid cleanup ---
    for col in ['paid_to_total_ratio', 'paid_to_avg_month_ratio', 'paid_to_last_month_ratio']:
        df[col] = df[col].replace([np.inf, -np.inf], np.nan)
        df[col] = df[col].fillna(df[col].median())

  
  
    return df



In [142]:
paid_cons_features = add_current_paid_consumption_features(df[['id','cons_12m','imp_cons','cons_last_month','elec_drop_flag']])
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,tenure_bucket,time_to_end_months,contract_expired_flag,time_to_renewal_months,renewal_missed_flag,avg_monthly_cons,elec_usage_ratio,elec_drop_flag,avg_monthly_gas,low_gas_user_flag
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.000000,...,very_loyal,0,1,0,1,0.000000,0.873417,0,4578.833333,0
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.950000,...,very_loyal,0,1,0,1,388.333333,0.000000,1,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.960000,...,very_loyal,0,1,0,1,45.333333,0.000000,1,0.000000,0


In [143]:
  

cols_to_merge = [c for c in ['id','paid_to_total_ratio', 'paid_to_avg_month_ratio',
                              'paid_to_last_month_ratio','payment_lag_flag','engagement_score']
                 if c in paid_cons_features.columns]

df = df.merge(paid_cons_features[cols_to_merge], on='id', how='left')


#df = df.merge(paid_cons_features[['id','paid_to_total_ratio', 'paid_to_avg_month_ratio', 'paid_to_last_month_ratio','payment_lag_flag','engagement_score']],
                           #on='id', how='left')
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,avg_monthly_cons,elec_usage_ratio,elec_drop_flag,avg_monthly_gas,low_gas_user_flag,paid_to_total_ratio,paid_to_avg_month_ratio,paid_to_last_month_ratio,payment_lag_flag,engagement_score
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.000000,...,0.000000,0.873417,0,4578.833333,0,0.000854,0.010249,0.096377,1,0.462452
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.950000,...,388.333333,0.000000,1,0.000000,0,0.000000,0.000000,0.096377,1,0.000000
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.960000,...,45.333333,0.000000,1,0.000000,0,0.000000,0.000000,0.096377,1,0.000000


In [144]:
def add_margin_features(df):
    """
    Create derived margin features for churn prediction.
    
    Input columns required:
        - margin_gross_pow_ele : gross margin on power subscription
        - margin_net_pow_ele   : net margin on power subscription
        - net_margin           : total net margin
    
    Returns:
        DataFrame with new engineered features:
            - margin_efficiency_pow_ele
            - margin_loss_pow_ele
            - pow_ele_contrib_to_total
            - margin_category
    """

    df = df.copy()

    # --- Basic checks ---
    required_cols = ['margin_gross_pow_ele', 'margin_net_pow_ele', 'net_margin']
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")

    # Replace zeros and infinities to avoid divide-by-zero
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # --- 1️⃣ Margin Efficiency (how much of gross margin is retained as net margin) ---
    df['margin_efficiency_pow_ele'] = df['margin_net_pow_ele'] / df['margin_gross_pow_ele']

    # --- 2️⃣ Margin Loss (difference between gross and net) ---
    df['margin_loss_pow_ele'] = df['margin_gross_pow_ele'] - df['margin_net_pow_ele']

    # --- 3️⃣ Power margin contribution to total net margin ---
    df['pow_ele_contrib_to_total'] = df['margin_net_pow_ele'] / df['net_margin']

    # --- 4️⃣ Margin Category (bucketed feature) ---
    df['margin_category'] = pd.cut(
        df['margin_net_pow_ele'],
        bins=[-float('inf'), 0, 50, 200, float('inf')],
        labels=['negative', 'low', 'medium', 'high']
    )

    # --- 5️⃣ Handle NaNs and invalid values ---
    for col in ['margin_efficiency_pow_ele', 'margin_loss_pow_ele', 'pow_ele_contrib_to_total']:
        df[col] = df[col].replace([np.inf, -np.inf], np.nan)
        df[col] = df[col].fillna(df[col].median())

    return df

In [145]:
#df = add_margin_features(df[['id','margin_gross_pow_ele','margin_net_pow_ele','net_margin']])

margin_features = add_margin_features(df[['id','margin_gross_pow_ele','margin_net_pow_ele','net_margin']])


cols_to_merge = ['id','margin_efficiency_pow_ele','margin_loss_pow_ele','pow_ele_contrib_to_total','margin_category']

df = df.merge(margin_features[cols_to_merge], on='id', how='left')

df.head(3)




,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,low_gas_user_flag,paid_to_total_ratio,paid_to_avg_month_ratio,paid_to_last_month_ratio,payment_lag_flag,engagement_score,margin_efficiency_pow_ele,margin_loss_pow_ele,pow_ele_contrib_to_total,margin_category
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.000000,...,0,0.000854,0.010249,0.096377,1,0.462452,1.000000,0.000000,0.037467,low
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.950000,...,0,0.000000,0.000000,0.096377,1,0.000000,1.000000,0.000000,0.867125,low
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.960000,...,0,0.000000,0.000000,0.096377,1,0.000000,1.000000,0.000000,4.333333,low


In [146]:
def add_forecast_features(df):
    """
    Create forecast-based features for churn prediction.

    Expected input columns:
        - forecast_cons_12m
        - forecast_cons_year
        - forecast_discount_energy
        - forecast_meter_rent_12m
        - forecast_price_energy_off_peak
        - forecast_price_energy_peak
        - forecast_price_pow_off_peak
    Optional (for comparison):
        - cons_12m               : past 12-month consumption
        - price_energy_off_peak  : current off-peak price
        - price_energy_peak      : current peak price
    """
    #pd.set_option('display.float_format', '{:.6f}'.format)
    df = df.copy()

    # --- Replace inf/NaN early ---
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # --- 1️⃣ Forecast average energy price ---
    df['forecast_avg_energy_price'] = df[
        ['forecast_price_energy_off_peak', 'forecast_price_energy_peak']
    ].mean(axis=1)

    # --- 2️⃣ Forecasted estimated annual energy cost ---
    df['forecast_estimated_energy_cost'] = (
        df['forecast_cons_12m'] * df['forecast_avg_energy_price']
    )

    # --- 3️⃣ Forecast vs past consumption ratio ---
    if 'cons_12m' in df.columns:
        df['forecast_vs_past_cons_ratio'] = (
            df['forecast_cons_12m'] / df['cons_12m'].replace(0, np.nan)
        )
        df['forecast_cons_change_flag'] = (df['forecast_vs_past_cons_ratio'] > 1.2).astype(int)
    else:
        df['forecast_vs_past_cons_ratio'] = np.nan
        df['forecast_cons_change_flag'] = np.nan

    # --- 4️⃣ Expected energy price increase (if current prices exist) ---
    if {'price_energy_off_peak', 'price_energy_peak'}.issubset(df.columns):
        df['current_avg_energy_price'] = df[
            ['price_energy_off_peak', 'price_energy_peak']
        ].mean(axis=1)
        df['expected_energy_price_increase'] = (
            (df['forecast_avg_energy_price'] - df['current_avg_energy_price'])
            / df['current_avg_energy_price'].replace(0, np.nan)
        )
    else:
        df['expected_energy_price_increase'] = np.nan

    # --- 5️⃣ Forecasted discount ratio ---
    df['forecast_discount_ratio'] = (
        df['forecast_discount_energy'] /
        df['forecast_estimated_energy_cost'].replace(0, np.nan)
    )

    # --- 6️⃣ Forecasted meter rent ratio ---
    df['forecast_meter_rent_ratio'] = (
        df['forecast_meter_rent_12m'] /
        (df['forecast_estimated_energy_cost'] + 1e-6)
    )

    # --- 7️⃣ Combined "forecast sensitivity" score ---
    df['expected_energy_price_increase_scaled'] = (
    df['expected_energy_price_increase'] / (df['expected_energy_price_increase'].mean() + 1e-3)
    ).fillna(0).clip(0, 1)

    df['forecast_meter_rent_ratio_scaled'] = (
    df['forecast_meter_rent_ratio'] / (df['forecast_meter_rent_ratio'].max() + 1e-6)
    ).fillna(0).clip(0, 1)

    df['forecast_discount_ratio_scaled'] = (
    df['forecast_discount_ratio'].fillna(0).clip(0, 1)
    )

    # Compute final score
    df['forecast_sensitivity_score'] = (
    0.4 * df['expected_energy_price_increase_scaled'] +
    0.3 * df['forecast_meter_rent_ratio_scaled'] +
    0.3 * (1 - df['forecast_discount_ratio_scaled'])
    )

    
    df['forecast_sensitivity_score'] = (df['forecast_sensitivity_score'] * 100).round(2)

    # --- 8️⃣ Clean up infinities and fill remaining NaNs ---
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)

    return df


In [147]:
forecast_features = add_forecast_features(df[['id','cons_12m','forecast_cons_12m'
        ,'forecast_cons_year'
        ,'forecast_discount_energy'
        ,'forecast_meter_rent_12m'
        ,'forecast_price_energy_off_peak'
        ,'forecast_price_energy_peak'
        ,'forecast_price_pow_off_peak']])


cols_to_merge = ['id','forecast_avg_energy_price',
    'forecast_estimated_energy_cost',
    'forecast_vs_past_cons_ratio',
    'expected_energy_price_increase',
    'forecast_discount_ratio',
    'forecast_meter_rent_ratio_scaled',
    'forecast_sensitivity_score']

df = df.merge(forecast_features[cols_to_merge], on='id', how='left')

df.head(3)


,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,margin_loss_pow_ele,pow_ele_contrib_to_total,margin_category,forecast_avg_energy_price,forecast_estimated_energy_cost,forecast_vs_past_cons_ratio,expected_energy_price_increase,forecast_discount_ratio,forecast_meter_rent_ratio_scaled,forecast_sensitivity_score
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.000000,...,0.000000,0.037467,low,0.106312,0.000000,0.000000,0.000000,0.000000,0.003167,30.090000
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.950000,...,0.000000,0.867125,low,0.072855,13.838902,0.040762,0.000000,0.000000,0.000000,30.000000
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.960000,...,0.000000,4.333333,low,0.126847,6.083558,0.088162,0.000000,0.000000,0.000000,30.000000


In [148]:
'''The input data really has no variation (e.g., all customers have the same forecast values).

Then, this feature won’t help churn prediction — we may skip it or mark it as low importance.'''

'The input data really has no variation (e.g., all customers have the same forecast values).\n\nThen, this feature won’t help churn prediction — we may skip it or mark it as low importance.'